In [5]:
import json
import pandas as pd
import numpy as np
import pickle

In [6]:
user_to_encode = "../processed_datas/val_unseen.json"
word_embeds = "../processed_datas/word_embeds.pkl"
out_file = f"{user_to_encode.rstrip('.json')}_embeds.pkl"

with open(user_to_encode, "r") as f:
    user_to_encode = json.load(f)

user_to_encode = pd.DataFrame(user_to_encode)
texts = user_to_encode["text"]

In [7]:
class counter():
    def __init__(self):
        self.count = {}
    def update(self, key):
        self.count[key] = self.count.get(key, 0) + 1
    def __str__(self):
        return str(self.count)

genders = counter()
occupations = counter()
interests = counter()
recreations = counter()
for text in texts:
    splits = [text.index("[OCCUPATION]"), text.index("[INTERESTS]"), text.index("[RECREATION]")]
    gender_text = text[:splits[0]][8:]
    occupation_text = text[splits[0]:splits[1]][12:]
    interests_text = text[splits[1]:splits[2]][11:]
    recreation_text = text[splits[2]:][12:]

    genders.update(gender_text)
    for o in occupation_text.split(","):
        occupations.update(o)
    for i in interests_text.split(","):
        interests.update(i)
    for r in recreation_text.split(","):
        recreations.update(r)

counts = {
    "gender": genders.count,
    "occupation": occupations.count,
    "interests": interests.count,
    "recreation": recreations.count
}

total_features = sum([len(counts[key]) for key in counts])
feats = []

for text in texts:
    splits = [text.index("[OCCUPATION]"), text.index("[INTERESTS]"), text.index("[RECREATION]")]
    gender_text = text[:splits[0]][8:]
    occupation_text = text[splits[0]:splits[1]][12:]
    interests_text = text[splits[1]:splits[2]][11:]
    recreation_text = text[splits[2]:][12:]

    feat = np.zeros(total_features)
    i = 0
    pos = list(counts["gender"].keys()).index(gender_text)
    feat[i + pos] = feat[i + pos] + 1

    i += len(counts["gender"].keys())
    for o in occupation_text.split(","):
        pos = list(counts["occupation"].keys()).index(o)
        feat[i + pos] = feat[i + pos] + 1
    
    i += len(counts["occupation"].keys())
    for it in interests_text.split(","):
        pos = list(counts["interests"].keys()).index(it)
        feat[i + pos] = feat[i + pos] + 1

    i += len(counts["interests"].keys())
    for r in recreation_text.split(","):
        pos = list(counts["recreation"].keys()).index(r)
        feat[i + pos] = feat[i + pos] + 1

    feats.append(feat)

In [8]:
with open(out_file, "wb") as f:
    pickle.dump(feats, f)

In [13]:
words = [k for c in counts.values() for k in c.keys()]
print(words)
print(len(words))

['female', 'other', 'male', '', '金融業', '服務業', '', '教學專業', '廣告傳播', '自由業', '製造業', '資訊科技', '農林漁牧', '非營利組織', '家管', '藝文設計', '其他', '法律、社會及文化專業', '科技業', '公務人員', '營建工程', '醫療', '出版業', '職業軍人', '退休', '生活品味_寵物', '手作_手作小物', '生活品味_親子教育', '手作_刺繡', '生活品味_烹飪料理與甜點', '攝影_商業攝影', '設計_平面設計', '設計_應用設計', '行銷_文案', '攝影_後製剪輯', '行銷_數位行銷', '行銷_社群行銷', '程式_程式入門', '程式_程式理財', '程式_網頁前端', '程式_資料科學', '投資理財_金融商品', '投資理財_理財', '投資理財_投資觀念', '程式_程式語言', '程式_網頁後端', '職場技能_效率提升', '職場技能_個人品牌經營', '語言_英文', '職場技能_資料彙整', '藝術_繪畫與插畫', '藝術_手寫字', '藝術_字體設計', '投資理財_量化交易', '投資理財_比特幣', '職場技能_創業', '生活品味_運動', '設計_動態設計', '職場技能_職場溝通', '設計_介面設計', '攝影_動態攝影', '攝影_影像創作', '攝影_影視創作', '職場技能_產品設計', '程式_網站架設', '設計_網頁設計', '投資理財_更多投資理財', '藝術_素描', '藝術_更多藝術', '音樂_樂器', '音樂_音樂創作', '攝影_攝影理論', '程式_遊戲開發', '音樂_音樂理論', '音樂_人聲', '藝術_電腦繪圖', '手作_更多手作', '程式_量化分析', '攝影_更多攝影', '語言_歐洲語言', '行銷_更多行銷', '人文_社會科學', '人文_文學', '語言_日文', '藝術_角色設計', '程式_手機程式開發', '生活品味_心靈成長與教育', '人文_更多人文', '職場技能_文書處理', '設計_更多設計', '生活品味_壓力舒緩', '手作_手工書', '語言_更多語言', '語言_韓文', '設計_設計理論', '程式_區塊鏈', '手作_手工印刷'

In [14]:
from sentence_transformers import SentenceTransformer
m = SentenceTransformer("shibing624/text2vec-base-chinese")

embeds = m.encode(words)
print(embeds.shape)

with open(word_embeds, "wb") as f:
    pickle.dump(embeds, f)


No sentence-transformers model found with name /home/b09901073/.cache/torch/sentence_transformers/shibing624_text2vec-base-chinese. Creating a new one with MEAN pooling.


(153, 768)


In [17]:
print(feats[10])
print(len(feats[0]))

[0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
153


In [18]:
print(genders)
print(occupations)
print(interests)
print(recreations)

{'female': 7461, 'other': 215, 'male': 3451, '': 495}
{'金融業': 231, '服務業': 568, '': 8069, '教學專業': 340, '廣告傳播': 298, '自由業': 463, '製造業': 354, '資訊科技': 499, '農林漁牧': 49, '非營利組織': 100, '家管': 148, '藝文設計': 676, '其他': 577, '法律、社會及文化專業': 117, '科技業': 300, '公務人員': 171, '營建工程': 117, '醫療': 249, '出版業': 82, '職業軍人': 29, '退休': 20}
{'生活品味_寵物': 825, '手作_手作小物': 1584, '生活品味_親子教育': 811, '手作_刺繡': 770, '生活品味_烹飪料理與甜點': 2157, '攝影_商業攝影': 1809, '設計_平面設計': 3072, '設計_應用設計': 1097, '行銷_文案': 1786, '攝影_後製剪輯': 2378, '行銷_數位行銷': 2415, '行銷_社群行銷': 1799, '程式_程式入門': 1360, '程式_程式理財': 681, '程式_網頁前端': 951, '程式_資料科學': 671, '投資理財_金融商品': 1621, '投資理財_理財': 2966, '投資理財_投資觀念': 2607, '程式_程式語言': 1217, '程式_網頁後端': 568, '職場技能_效率提升': 1744, '職場技能_個人品牌經營': 1771, '語言_英文': 3380, '職場技能_資料彙整': 1812, '藝術_繪畫與插畫': 3049, '藝術_手寫字': 1310, '藝術_字體設計': 1085, '投資理財_量化交易': 548, '投資理財_比特幣': 526, '職場技能_創業': 1377, '生活品味_運動': 1433, '設計_動態設計': 1341, '職場技能_職場溝通': 2014, '設計_介面設計': 1047, '攝影_動態攝影': 1186, '攝影_影像創作': 2739, '攝影_影視創作': 922, '職場技能_產品設計': 1022, '程式_網站架設': 7